# Introduction

This is the first time that I particpate in a computer vision related competition, and boy it is hard. I don't have much background information to tackle that task at hand, and the stream of information I'm trying to ingest is HUGE!

So I decided to take a detour from the required objective and try something else that I can get more easily familiar with, and I hope that it can still translate into more understanding of the competition, and give me a little bit of confidence to jump into the main task of the competition.

Without further ado, let's get started.

In [ ]:
!pip install --user torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 torchtext==0.10.0
from fastai.vision.all import *
%config Completer.use_jedi = False

In [ ]:
df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.species.nunique()

In [ ]:
df.species.value_counts().sort_index()

In [ ]:
df.loc[:, 'species'] = df.species.replace({'bottlenose_dolpin': 'bottlenose_dolphin',
                                           'kiler_whale': 'killer_whale'})

In [ ]:
df.species.value_counts().sort_index()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
df['species'].value_counts().sort_values(ascending=True).plot(kind='barh');

## How can I make the validation set?

Well I guess that depends on the task. The task that I made up here is species classification, and if this data reflects the distribution of real life whales and dolphins, then the validation set should also have a similar distribution.

For example, by looking into the wikipedia entry of [frasers dolphins](https://en.wikipedia.org/wiki/Fraser%27s_dolphin), we can see that their sightings are rare, and therefore their small value in the dataset is justified.

Anyways, I'll use fastai's RandomSplitter to generate a validation set on the fly, and to make a fast prototype, I shall make another sample of the dataset and call it the dev set, which should allow fast training to test out any new ideas.

In [ ]:
from sklearn.model_selection import train_test_split

_, dev, _ , _ = train_test_split(df, df['species'], test_size=0.1)

dev.shape

Now let's use fastai datablock api to get us off the ground fast.

In [ ]:
path = Path("../input/jpeg-happywhale-256x256/train_images-256-256/train_images-256-256")

dev['image_path'] = dev['image'].apply(lambda f: path/f)


dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                   get_x=ColReader('image_path'),
                   get_y=lambda r: r['species'],
                   splitter=RandomSplitter(seed=42),
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224))
# dblock.summary(dev)
dsets = dblock.datasets(dev)
dls = dblock.dataloaders(dev)

In [ ]:
dls.show_batch(max_n=20)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=[accuracy, error_rate])
learn.lr_find()

In [ ]:
learn.fine_tune(2, base_lr=3e-3)

The model seems to be improving, so let's bump it up with some more epochs for the head.

In [ ]:
learn.freeze()
learn.fit_one_cycle(3, 3e-3)

Now let's give some love to the pretrained layers.

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, lr_max=slice(1e-6, 1e-4))

So it seems that we are bound to around 91% accuracy using this architecture and a 9% subset of the data for training. Which isn't bad given that we have conjured this up in a couple of minutes.

## Possible Use Case

I think that if in case of using metric learning and knn for selecting the top 5 similar images, one could use an accurate enough classifier to classify the current image, and match it only with other images of the same classification.

This could possibly be a use case for this kind of model.


Anyways, thanks for reading.